# Crawl data từ trang web bằng BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
# list chứa các link
all_linked_to_house = []

In [ ]:
# Lấy link 5000 bất động sản ở thành phố Hồ Chí Minh (từ trang 1 đến trang 250)
# và thêm vào list
for i in range(1, 251):
    page_to_scrape = requests.get(f'https://batdongsan.vn/ban-nha-ho-chi-minh/p{i}')
    soup = BeautifulSoup(page_to_scrape.text, 'lxml')
    houses = soup.find_all('div', class_ = 'datalist')[0].find_all('div', class_ = 'item')
    for house in houses:
        linked_to = house.find('a', class_ = '').get('href')
        all_linked_to_house.append(linked_to)

In [ ]:
# Tạo file csv để lưu 5000 link vừa lấy và đưa lên github
pd.DataFrame(all_linked_to_house).to_csv('all_linked_to_hcm_house.csv', index = False)

In [ ]:
# Load file 5000 link
df_link = pd.read_csv('https://raw.githubusercontent.com/loctv1705/project_nmkhdl/main/all_linked_to_hcm_house.csv').values.tolist()
df_link_1d = [item for sublist in df_link for item in sublist]

In [ ]:
# hàm sửa giá (price) nhà
def change(price):
    if 'tỷ' in price:
        price = price.replace(' tỷ', '')
        new_price = float(price) * int(1e9) # Chuyển giá chứa chữ 'tỷ' thành số đơn vị đồng
    elif 'triệu' in price:
        price = price.replace(' triệu', '')
        new_price = float(price) * int(1e6) # Chuyển giá chứa chữ 'triệu' thành số đơn vị đồng
    else:
        return 0
    return new_price

# trích xuất thông tin từ mô tả
def extract_info(thong_tin, key, unit=None):
    start = thong_tin.find(key)
    if start == -1:
        return None
    start += len(key)
    if unit:
        end = thong_tin.find(unit, start)
        if end == -1:
            return None
        value = thong_tin[start:end].strip()
    else:
        end = thong_tin.find('\n', start)
        if end == -1:
            value = thong_tin[start:].strip()
        else:
            value = thong_tin[start:end].strip()
    return value if unit else value

In [ ]:
# List để chứa các thông tin cần lưu
prices = []
areas = []
wcs = []
bedrooms = []
addresses = []
des_titles = []

In [ ]:
new_link_list = df_link_1d[:]

In [ ]:
# cào dữ liệu từ 1 nhà
for i, link in enumerate(new_link_list):
    scrape_html = requests.get(link)
    soup = BeautifulSoup(scrape_html.text, 'lxml')
    house_info = soup.find_all('div', class_ = 'uk-panel')[1]

    price = house_info.find('strong', class_ = 'price') # giá nhà
    if price == None:
        continue
    price = change(price.text)
    prices.append(price)

    title = house_info.find('h1', class_ = 'uk-panel-title').text # tiêu đề của bài đăng

    more_info = house_info.find('ul', class_ = 'uk-list').find_all('li') # các thông tin chính

    a_house = []
    for info in more_info:
        a_house.append(info.text)
    res = '\n'.join(a_house)
    area = extract_info(res, 'Diện tích: ', 'm2')
    areas.append(area)

    wc = extract_info(res, 'Phòng WC: ', 'WC')
    wcs.append(wc)

    bedroom = extract_info(res, 'Phòng ngủ: ', 'PN')
    bedrooms.append(bedroom)

    address = extract_info(res, 'Địa chỉ: ', None)
    try:
        addresses.append(address.upper())
    except:
        addresses.append(None)

    house_description = soup.find('div', class_ = 'project-global-object-block-003 block-custom').find('div', class_ = 'content') # mô tả nhà

    des_titles.append(title + '\n' + house_description.text)
    print(i)

In [ ]:
# Tạo biến tạm để lưu nhằm tránh mất dữ liệu gốc
price_t = prices
area_t = areas
bedroom_t = bedrooms
bathroom_t = wcs

In [ ]:
# Đổi kiểu dữ liệu
price_new = [x / 1000000000 for x in price_t] # Chuyển lại giá thành đơn vị tỷ
area_new = [float(x) if x is not None else None for x in area_t] # chuyển diện tích thành kiểu số thực
bedroom_new = [int(x) if x is not None else None for x in bedroom_t] # chuyển thành số nguyên
bathroom_new = [int(x) if x is not None else None for x in bathroom_t] # chuyển thành số nguyên

In [ ]:
# Lưu các thông tin vừa cào vào dataframe và chuyển thành file csv
import pandas as pd

df = pd.DataFrame({'description_title': des_titles, 'price': price_new, 'area': area_new, 'bedroom': bedroom_new, 'bathroom': bathroom_new, 'address': addresses})

In [ ]:
df.to_csv('house_description.csv', index = False)

Sau khi lưu file csv thì nhóm em đã đưa file lên github để thuận tiện cho việc sử dụng về sau

# Sử dụng LLM để trích xuất thông tin từ đoạn mô tả nhà.

In [ ]:
import os
import requests

s = requests.Session()
# Sử dụng API LLM model trên web anyscale
api_base = "https://api.endpoints.anyscale.com/v1"
# Token này sẽ được thay đổi sau một khoảng thời gian chạy
token = "esecret_hytywgz6a7g7fvc43pecgu99gd"
url = f"{api_base}/chat/completions"

In [ ]:
import re
import json

In [ ]:
list_feature=[] # danh sách dùng để lưu những thông tin sau của mỗi căn nhà sau khi chạy LLM
count=0

In [ ]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/loctv1705/project_nmkhdl/main/house_description.csv") # đọc file csv sau khi đã crawl data

In [ ]:
df

,description_title,price,area,bedroom,bathroom,address
0,Chưa tới 30tr/m2 - Hàng ngộp bank BAO ĐẦU TƯ ...,3.899000e+09,150.0,2.0,1.0,THÀNH PHỐ HỒ CHÍ MINH
1,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",5.500000e+00,51.0,NaN,NaN,ÂU CƠ PHƯỜNG 9 TÂN BÌNH
2,"SÁT MẶT TIỀN PHAN ĐĂNG LƯU, PHƯỜNG 7, PHÚ NHUẬ...",4.600000e+00,45.0,2.0,2.0,NaN
3,CHỦ GẤP BÁN TRƯỚC TẾT LÊ HỒNG PHONG QUẬN 5 RA ...,7.350000e+00,41.0,NaN,NaN,LÊ HỒNG PHONG PHƯỜNG 2 QUẬN 5
4,"LŨY BÁN BÍCH,TÂN PHÚ-DIỆN TÍCH KHỦNG 96M2 ( 4....",0.000000e+00,96.0,2.0,1.0,NaN
...,...,...,...,...,...,...
4995,Nhà mới đẹp lung linh - 5 tầng - 30m2 - nhỉnh ...,0.000000e+00,30.0,3.0,3.0,NaN
4996,"Nhà VIP, HXH 10m ra MTĐ số 8 khu phố Linh Xuân...",3.050000e+06,100.0,NaN,NaN,NaN
4997,"HXH thông, Nguyễn Văn Khối Phường 8 Gò Vấp, 34...",4.500000e+00,34.0,2.0,2.0,NGUYỄN VĂN KHỐI
4998,Hẻm 10m Đặng Thuỳ Trâm 4x15 8.6 tỷ P13 Bình Th...,8.600000e+00,60.0,4.0,5.0,NaN


In [ ]:
def get_answer(question):
  body = {
  "model": "meta-llama/Meta-Llama-3-70B-Instruct",
  "messages": [
    {
      "role": "system",
      "content": "you are a helpful assistant and you are a real estate specialist"
    },
    {
      "role": "user",
      "content": f"{question}"
    }
  ],
  "temperature": 1,
  "max_tokens": 256,
  "top_p": 1,
  "frequency_penalty": 0
  }
  with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
    response=resp.json()
    answer=response['choices'][0]['message']['content']
    return answer
def get_json(answer): # Hàm dùng để trích xuất câu trả lời mong muốn nằm trong '{}' và sau đó chuyển đổi thành dạng List.
  match = re.search(r"\{[^}]+\}", answer, re.DOTALL)
  if match:
    json_string = match.group(0)
    print(json_string)
    # Thay thế NaN bằng null để biến nó thành định dạng JSON đúng
    json_string = json_string.replace("NaN", "null")
    # Thay thế True/False bằng true/false.
    json_string = json_string.replace("False", "false").replace("True", "true")
    try:
        # chuyển định dạng JSON thành dictionary trong python
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        print("\nFailed to parse JSON string:")
        print(e)
  else:
    print("No JSON object found in the text.")
  return data

In [ ]:
for description in df['description_title']:
    # Tạo prompt câu hỏi
    question=f"""I have a description about house:{description}.
    Do not comment, do not provide explanations and do not add any extra features.
    Do not input any data that isn't like stated formats. Do not add anything into the input to make it fit into stated formats.
    Ignore remaining data that doesn't have anything to do with the stated features.
    Extract information of description as json format:
    district: contains 'Quận' or 'Huyện', case insensitive, only pick the first value, data that don't contain 'Quận' or 'Huyện' should be treated as values that don't fit the format (values that don't fit the format: null);
    price: float (null);
    area: float (null);
    bedroom: int (null);
    bathroom: int (null);
    facade: bool (null);
    nearhospital: bool (null);
    nearschool: bool (null);
    nearsupermarket: bool (null);
    nearuniversity: bool (null);
    nearbuildings: bool (null);
    urgent: bool (null),
    floor: int (null)
    """
    answer=get_answer(question) # Lấy câu trả lời từ LLM
    try:
      json_string=get_json(answer)  # Chuyển câu trả lời sang dạn List
      list_feature.append(json_string)  # Thêm các đặc trưng thu được vào List
      print("complete number:",count)
      count+=1  # Đếm số lượng đã thực hiện được
    except:
      print('error at: ',count) # Sẽ có một số mô tả không phải là của nhà bán thì LLM sẽ không có được định dạng trả lời như mong muốn nên sẽ bị lỗi
      list_feature.append(json.loads("{}")) # Những câu trả lời bị lỗi sẽ được lưu lại là List trống và sẽ được sử lý trong bước Preprocessing
      count+=1
      continue


In [ ]:
n=pd.DataFrame(list_feature) # Chuyển sang file dataframe

In [ ]:
n=n.iloc[:,0:13]
# Trong quá trình trích xuất đặc trưng của LLM, sẽ có một số feature bị thay tên như là: 'nearschool' đổi thành 'nears_chool'. Hoặc là sẽ tạo thêm những feature mới không cần thiết nên chỉ lấy những feature chúng ta đưa ra ban đầu

In [ ]:
n

,district,price,area,bedroom,bathroom,facade,nearhospital,nearschool,nearsupermarket,nearuniversity,nearbuildings,urgent,floor
0,Huyện Nhà Bè,30.0,150.0,2.0,1.0,TRUE,NaN,NaN,NaN,NaN,NaN,True,NaN
1,Tân Bình,5.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2,Phú Nhuan,4.0,45.0,2.0,2.0,TRUE,True,NaN,True,NaN,NaN,NaN,2.0
3,Quận 5,NaN,41.0,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN,True,2.0
4,Tân Phú,5.0,96.0,NaN,NaN,NaN,NaN,True,NaN,True,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,NaN,4.0,30.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4996,Thủ Đức,3.0,100.0,2.0,2.0,True,NaN,True,True,NaN,NaN,NaN,NaN
4997,Gò Vấp,4.5,34.0,2.0,2.0,True,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
n.to_csv('description_llm.csv',index=False) # Lưu lại thành file csv

Tương tự như trên, nhóm em cũng đưa file này lên github

- Lưu ý: Trong quá trình crawl data và trích xuất thông tin từ LLM thì chúng em chia nhau những phần để thực hiện cho nhanh, đoạn code trên đây là cách nhìn tổng thể, trong quá trình mỗi bạn chạy chỉ thay đổi chỉ số index và sau đó gộp lại đúng theo thứ tự, không có sự thay đổi về chất lượng hay số lượng.